In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from torch.optim import Adam
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

In [34]:
class TemporalConvLayer(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(TemporalConvLayer, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=1)
        self.conv2 = nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1)

    def forward(self, x):
        # x shape: (batch_size, num_nodes, num_features)
        x = x.permute(0, 2, 2)  # Change to (batch_size, num_features, num_nodes)
        x1 = self.conv1(x)
        x2 = self.conv2(x)
        return F.relu(x1 + x2).permute(0, 2, 2)

class STGCN(nn.Module):
    def __init__(self, node_features, temporal_features, spatial_features):
        super(STGCN, self).__init__()
        self.temporal1 = TemporalConvLayer(node_features, temporal_features)
        self.spatial1 = GCNConv(temporal_features, spatial_features)
        self.temporal2 = TemporalConvLayer(spatial_features, spatial_features)
        self.spatial2 = GCNConv(spatial_features, spatial_features)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.temporal1(x)
        x = F.relu(self.spatial1(x, edge_index))
        x = self.temporal2(x)
        x = F.relu(self.spatial2(x, edge_index))
        return x

In [35]:
df = pd.read_csv('../../Datasets/Lynx_Hare/data.csv', usecols=['DATE', 'HARE', 'LYNX'])
scaler = StandardScaler()
df[['HARE', 'LYNX']] = scaler.fit_transform(df[['HARE', 'LYNX']])

df['HARE_lag1'] = df['HARE'].shift(1)
df['LYNX_lag1'] = df['LYNX'].shift(1)
df.dropna(inplace=True)  
print(df)
features = torch.tensor(df[['HARE_lag1', 'LYNX_lag1']].values, dtype=torch.float)
targets = torch.tensor(df[['HARE', 'LYNX']].values, dtype=torch.float)
edge_index = torch.tensor([[0], [1]], dtype=torch.long) 


    DATE      HARE      LYNX  HARE_lag1  LYNX_lag1
1   1846 -0.713379  0.896212  -0.713932   0.093463
2   1847 -0.713103  1.109426  -0.713379   0.896212
3   1848 -0.923746  0.596114  -0.713103   1.109426
4   1849 -0.480069 -0.378806  -0.923746   0.596114
5   1850  0.348128 -1.061622  -0.480069  -0.378806
..   ...       ...       ...        ...        ...
86  1931 -0.715591 -1.067486  -1.138258  -1.138379
87  1932  1.014610 -0.657049  -0.715591  -1.067486
88  1933  1.226082 -0.187446   1.014610  -0.657049
89  1934  1.002171  0.072674   1.226082  -0.187446
90  1935 -0.819530  0.376504   1.002171   0.072674

[90 rows x 5 columns]


In [36]:
indices = list(range(features.shape[0]))
train_indices, test_indices = train_test_split(indices, test_size=0.2, random_state=42)

data_train = Data(x=features[train_indices], edge_index=edge_index)
data_test = Data(x=features[test_indices], edge_index=edge_index)

model = STGCN(node_features=2, temporal_features=64, spatial_features=64)
optimizer = Adam(model.parameters(), lr=0.01)
criterion = nn.MSELoss()

In [37]:
train_losses = []
test_losses = []

for epoch in range(200):
    model.train()
    optimizer.zero_grad()
    out = model(data_train)
    train_loss = criterion(out, data_train.x)
    train_loss.backward()
    optimizer.step()
    
    model.eval()  
    with torch.no_grad(): # gradient is not computed
        test_pred = model(data_test)
        test_loss = criterion(test_pred, data_test.x)
    
    train_losses.append(train_loss.item())
    test_losses.append(test_loss.item())

# training and testing losses
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss')
plt.plot(test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Test Loss Over Epochs')
plt.legend()
plt.show()

model.eval() 
with torch.no_grad(): 
    final_pred = model(data_test)
    final_mse_test = criterion(final_pred, data_test.x)

print(f'Final Test MSE: {final_mse_test.item()}')

IndexError: Dimension out of range (expected to be in range of [-2, 1], but got 2)